In [1]:
# This is a program that trains a ResNet-18 model on Tiny-ImageNet dataset


# import necessary libraries
import torch 
import torchvision

import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import models, utils, datasets, transforms

import numpy as np

import sys
import os
from PIL import Image

import matplotlib.pyplot as plt

In [2]:
# set device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# set hyperparameters
EPOCH = 3
pre_epoch = 0
BATCH_SIZE = 64
LR = 0.01

In [4]:
# prepare Tiny-ImageNet dataset and preprocess it
transform_train = transforms.Compose([
    transforms.RandomCrop(64, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4802, 0.4481, 0.3975), (0.2302, 0.2265, 0.2262))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4802, 0.4481, 0.3975), (0.2302, 0.2265, 0.2262))
])

trainset = datasets.ImageFolder(root='/Users/royalty/Desktop/Python_ML/Python-DL/second_assignment/tiny-imagenet-200/train', transform=transform_train)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = datasets.ImageFolder(root='/Users/royalty/Desktop/Python_ML/Python-DL/second_assignment/tiny-imagenet-200/val', transform=transform_test)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [5]:
# import a ResNet-18 model
net = models.resnet18(pretrained=False, num_classes=200)
net = net.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

/Users/royalty/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/royalty/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
# write a function to train the model
def train(epoch):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # print loss every 50 batches
        if batch_idx % 50 == 0:
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

        
# write a function to test the model
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            # print loss every 50 batches
            if batch_idx % 50 == 0:
                print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [9]:
# train the model
for epoch in range(pre_epoch, pre_epoch + EPOCH):
    train(epoch)
    test(epoch)


Epoch: 1
Loss: 5.409 | Acc: 0.000% (0/64)
Loss: 5.416 | Acc: 0.613% (20/3264)
Loss: 5.359 | Acc: 0.804% (52/6464)
Loss: 5.295 | Acc: 1.076% (104/9664)
Loss: 5.242 | Acc: 1.415% (182/12864)


KeyboardInterrupt: 

In [ ]:
# save the model
torch.save(net.state_dict(), 'resnet18.pth')